# Market Data & Trade Analysis Challenge

In this notebook, I am provided with the market data and trade data for four hypothetical instruments. For each instrument, I am required to deliver:
1. A plot visualizing both the market data and the trade data, for an interesting one hour period of my choice. 
2. a plot visualizing the aggregate markout curve of all trades for the instrument, using the intervals specified in the file "horizon_ticks". Please ensure your curve shows "margin" - PNL normalized by volume.


### Part A: Data Cleaning, PnL Computation, and Exploratory Data Analysis

First, I will make sure that the data provided is suitable for analysis. I will clean the data, compute the cumulative PnL and markout PnL for each instrument, and perform some exploratory data analysis to understand the data better.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

!pip install pandas
!pip install numpy
!pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
metadata_files = {
    "BTAUSD": "data/BTAUSD.json",
    "GMMAUSD": "data/GMMAUSD.json",
    "LMDAUSD": "data/LMDAUSD.json",
    "ZTAUSD": "data/ZTAUSD.json",
}

metadata = {}
for symbol, file_path in metadata_files.items():
    with open(file_path, "r") as f:
        metadata[symbol] = json.load(f)

for symbol, data in metadata.items():
    print(f"{symbol} Metadata: {data}")


The meta-data confirms we have tick-level market data.

In [ ]:
data_files = {
    "BTAUSD_md": "data/BTAUSD_md.csv",
    "BTAUSD_trades": "data/BTAUSD_trades.csv",

    "GMMAUSD_md": "data/GMMAUSD_md.csv",
    "GMMAUSD_trades": "data/GMMAUSD_trades.csv",

    "LMDAUSD_md": "data/LMDAUSD_md.csv",
    "LMDAUSD_trades": "data/LMDAUSD_trades.csv",

    "ZTAUSD_md": "data/ZTAUSD_md.csv",
    "ZTAUSD_trades": "data/ZTAUSD_trades.csv",
}

market_data = {}
trade_data = {}

for key, file_path in data_files.items():
    if "_md" in key:
        market_data[key.replace("_md", "")] = pd.read_csv(file_path)
    elif "_trades" in key:
        trade_data[key.replace("_trades", "")] = pd.read_csv(file_path)

In [ ]:
for symbol in market_data:
  print(f"Processing symbol {symbol}...")
  print(market_data[symbol].head(3))

In [ ]:
for symbol in trade_data:
  print(f"Processing symbol {symbol}...")
  print(trade_data[symbol].head(3))

In [ ]:
for symbol, df in market_data.items():
    market_na = df.isna().any(axis=1)
    print(f"Market Data NA values for {symbol}: {sum(market_na)}")

for symbol, df in trade_data.items():
    trade_na = df.isna().any(axis=1)
    print(f"Trade Data NA values for {symbol}: {sum(trade_na)}")

In [ ]:
# if you try to run this cell twice it will fail because we add a new column
# 'mid_price' to market data
for symbol in market_data:
    # mental note: timestamp is in miliseconds
    market_data[symbol].columns = ["timestamp", "bid", "ask"] 
    market_data[symbol]["mid_price"] = (market_data[symbol]["bid"] 
                                        + market_data[symbol]["ask"]) / 2

for symbol in trade_data:
    trade_data[symbol].columns = ["timestamp", "trade_id", "side", "size", "price", "lhs_ccy", "rhs_ccy"]

In [ ]:
merged_data = {}

for symbol in market_data:
    merged_data[symbol] = pd.merge_asof(
        trade_data[symbol].sort_values("timestamp"),
        market_data[symbol].sort_values("timestamp"),
        on="timestamp",
        direction="forward" # interpretation: last book seen before trade
        # direction="backward" # interpretation: effective book immediately after trade
    )

for symbol in merged_data:
    print(f"Processing symbol {symbol}")
    print(merged_data[symbol].head(3))

Here, I decided to merge with `direction` set to `forward`. The interpretation for this merge result is that the `bid`, `ask`, and `mid_price` will be the last book seen before trade. On the other hand, we could have also merged with `direction` set to `backward`. The interpretation for this merge result is that the `bid`, `ask`, and `mid_price` will be the first book seen after trade.

The reason I decided to merge with `direction` set to `forward` is because this interpretation makes more sense to me.

In [ ]:
def compute_pnl(trade_df):
    trade_df["signed_size"] = trade_df["size"] * np.where(trade_df["side"] == "B", 1, -1)
    trade_df["open_position"] = trade_df["signed_size"] * trade_df["mid_price"]
    trade_df["fill_value"] = trade_df["signed_size"] * trade_df["price"]
    trade_df["pnl"] = trade_df["open_position"] - trade_df["fill_value"]
    trade_df["cumulative_pnl"] = trade_df["open_position"].cumsum() - trade_df["fill_value"].cumsum()

    return trade_df

for symbol in merged_data:
    print(f"Computing pnl for symbol {symbol}")
    compute_pnl(merged_data[symbol])
    print(merged_data[symbol].head())

The formula I used to calculate the cumulative profit and loss (PnL) of the trading strategy is as follows:

$\text{Cumulative PnL} = \sum q_i m - \sum q_i p_i$

where:
- \( q \) is the quantity traded (positive for buys, negative for sells).
- \( m \) is the market price (mid-price at a given time).
- \( p \) is the execution/fill price of the trade.
- The first summation represents the open position value (quantity times mid-price).
- The second summation represents the total invested capital (quantity times trade price).

In [ ]:
for symbol, symbol_df in merged_data.items():
    df = symbol_df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    fig, ax1 = plt.subplots(figsize=(10, 5))

    ax1.set_xlabel("Timestamp")
    ax1.set_ylabel("PnL", color="tab:blue")
    ax1.plot(df["timestamp"], df["pnl"], label="PnL", color="tab:blue")
    ax1.tick_params(axis="y", labelcolor="tab:blue")

    ax2 = ax1.twinx()
    ax2.set_ylabel("Cumulative PnL", color="tab:red")
    ax2.plot(df["timestamp"], df["cumulative_pnl"], label="Cumulative PnL", color="tab:red")
    ax2.tick_params(axis="y", labelcolor="tab:red")

    plt.title(f"PnL and Cumulative PnL for {symbol}")
    fig.tight_layout()
    plt.show()


For `GMMAUSD` and `ZTAUSD` the cumulative pnl curve trends upward while for `BTAUSD` and `LMDAUSD`, the cumulative pnl curve trends downward over time. Furthermore, all instruments show spikes between timestamp `1 May 6pm` and `2 May 6am.`, which could indicate key market events or inefficiencies in the trade execution.

In [ ]:
# all in ms
horizon_ticks = [-10000, -9900, -9800, -9700, -9600, -9500, -9400, -9300, -9200, -9100, -9000, -8900, -8800, -8700, -8600, -8500, -8400, -8300, -8200, -8100, -8000, -7900, -7800, -7700, -7600, -7500, -7400, -7300, -7200, -7100, -7000, -6900, -6800, -6700, -6600, -6500, -6400, -6300, -6200, -6100, -6000, -5900, -5800, -5700, -5600, -5500, -5400, -5300, -5200, -5100, -5000, -4900, -4800, -4700, -4600, -4500, -4400, -4300, -4200, -4100, -4000, -3900, -3800, -3700, -3600, -3500, -3400, -3300, -3200, -3100, -3000, -2900, -2800, -2700, -2600, -2500, -2400, -2300, -2200, -2100, -2000, -1900, -1800, -1700, -1600, -1500, -1400, -1300, -1200, -1100, -1000, -900, -800, -700, -600, -500, -400, -300, -200, -100, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800, 9900, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000, 26000, 27000, 28000, 29000, 30000, 31000, 32000, 33000, 34000, 35000, 36000, 37000, 38000, 39000, 40000, 41000, 42000, 43000, 44000, 45000, 46000, 47000, 48000, 49000, 50000, 51000, 52000, 53000, 54000, 55000, 56000, 57000, 58000, 59000, 60000, 61000, 62000, 63000, 64000, 65000, 66000, 67000, 68000, 69000, 70000, 71000, 72000, 73000, 74000, 75000, 76000, 77000, 78000, 79000, 80000, 81000, 82000, 83000, 84000, 85000, 86000, 87000, 88000, 89000, 90000, 91000, 92000, 93000, 94000, 95000, 96000, 97000, 98000, 99000, 100000, 101000, 102000, 103000, 104000, 105000, 106000, 107000, 108000, 109000, 110000, 111000, 112000, 113000, 114000, 115000, 116000, 117000, 118000, 119000, 120000, 121000, 122000, 123000, 124000, 125000, 126000, 127000, 128000, 129000, 130000, 131000, 132000, 133000, 134000, 135000, 136000, 137000, 138000, 139000, 140000, 141000, 142000, 143000, 144000, 145000, 146000, 147000, 148000, 149000, 150000, 151000, 152000, 153000, 154000, 155000, 156000, 157000, 158000, 159000, 160000, 161000, 162000, 163000, 164000, 165000, 166000, 167000, 168000, 169000, 170000, 171000, 172000, 173000, 174000, 175000, 176000, 177000, 178000, 179000, 180000, 181000, 182000, 183000, 184000, 185000, 186000, 187000, 188000, 189000, 190000, 191000, 192000, 193000, 194000, 195000, 196000, 197000, 198000, 199000, 200000, 201000, 202000, 203000, 204000, 205000, 206000, 207000, 208000, 209000, 210000, 211000, 212000, 213000, 214000, 215000, 216000, 217000, 218000, 219000, 220000, 221000, 222000, 223000, 224000, 225000, 226000, 227000, 228000, 229000, 230000, 231000, 232000, 233000, 234000, 235000, 236000, 237000, 238000, 239000, 240000, 241000, 242000, 243000, 244000, 245000, 246000, 247000, 248000, 249000, 250000, 251000, 252000, 253000, 254000, 255000, 256000, 257000, 258000, 259000, 260000, 261000, 262000, 263000, 264000, 265000, 266000, 267000, 268000, 269000, 270000, 271000, 272000, 273000, 274000, 275000, 276000, 277000, 278000, 279000, 280000, 281000, 282000, 283000, 284000, 285000, 286000, 287000, 288000, 289000, 290000, 291000, 292000, 293000, 294000, 295000, 296000, 297000, 298000, 299000, 300000, 301000, 302000, 303000, 304000, 305000, 306000, 307000, 308000, 309000, 310000, 311000, 312000, 313000, 314000, 315000, 316000, 317000, 318000, 319000, 320000, 321000, 322000, 323000, 324000, 325000, 326000, 327000, 328000, 329000, 330000, 331000, 332000, 333000, 334000, 335000, 336000, 337000, 338000, 339000, 340000, 341000, 342000, 343000, 344000, 345000, 346000, 347000, 348000, 349000, 350000, 351000, 352000, 353000, 354000, 355000, 356000, 357000, 358000, 359000, 360000, 361000, 362000, 363000, 364000, 365000, 366000, 367000, 368000, 369000, 370000, 371000, 372000, 373000, 374000, 375000, 376000, 377000, 378000, 379000, 380000, 381000, 382000, 383000, 384000, 385000, 386000, 387000, 388000, 389000, 390000, 391000, 392000, 393000, 394000, 395000, 396000, 397000, 398000, 399000, 400000, 401000, 402000, 403000, 404000, 405000, 406000, 407000, 408000, 409000, 410000, 411000, 412000, 413000, 414000, 415000, 416000, 417000, 418000, 419000, 420000, 421000, 422000, 423000, 424000, 425000, 426000, 427000, 428000, 429000, 430000, 431000, 432000, 433000, 434000, 435000, 436000, 437000, 438000, 439000, 440000, 441000, 442000, 443000, 444000, 445000, 446000, 447000, 448000, 449000, 450000, 451000, 452000, 453000, 454000, 455000, 456000, 457000, 458000, 459000, 460000, 461000, 462000, 463000, 464000, 465000, 466000, 467000, 468000, 469000, 470000, 471000, 472000, 473000, 474000, 475000, 476000, 477000, 478000, 479000, 480000, 481000, 482000, 483000, 484000, 485000, 486000, 487000, 488000, 489000, 490000, 491000, 492000, 493000, 494000, 495000, 496000, 497000, 498000, 499000, 500000, 501000, 502000, 503000, 504000, 505000, 506000, 507000, 508000, 509000, 510000, 511000, 512000, 513000, 514000, 515000, 516000, 517000, 518000, 519000, 520000, 521000, 522000, 523000, 524000, 525000, 526000, 527000, 528000, 529000, 530000, 531000, 532000, 533000, 534000, 535000, 536000, 537000, 538000, 539000, 540000, 541000, 542000, 543000, 544000, 545000, 546000, 547000, 548000, 549000, 550000, 551000, 552000, 553000, 554000, 555000, 556000, 557000, 558000, 559000, 560000, 561000, 562000, 563000, 564000, 565000, 566000, 567000, 568000, 569000, 570000, 571000, 572000, 573000, 574000, 575000, 576000, 577000, 578000, 579000, 580000, 581000, 582000, 583000, 584000, 585000, 586000, 587000, 588000, 589000, 590000, 591000, 592000, 593000, 594000, 595000, 596000, 597000, 598000, 599000, 600000, 601000, 602000, 603000, 604000, 605000, 606000, 607000, 608000, 609000, 610000, 611000, 612000, 613000, 614000, 615000, 616000, 617000, 618000, 619000, 620000, 621000, 622000, 623000, 624000, 625000, 626000, 627000, 628000, 629000, 630000, 631000, 632000, 633000, 634000, 635000, 636000, 637000, 638000, 639000, 640000, 641000, 642000, 643000, 644000, 645000, 646000, 647000, 648000, 649000, 650000, 651000, 652000, 653000, 654000, 655000, 656000, 657000, 658000, 659000, 660000, 661000, 662000, 663000, 664000, 665000, 666000, 667000, 668000, 669000, 670000, 671000, 672000, 673000, 674000, 675000, 676000, 677000, 678000, 679000, 680000, 681000, 682000, 683000, 684000, 685000, 686000, 687000, 688000, 689000, 690000, 691000, 692000, 693000, 694000, 695000, 696000, 697000, 698000, 699000, 700000, 701000, 702000, 703000, 704000, 705000, 706000, 707000, 708000, 709000, 710000, 711000, 712000, 713000, 714000, 715000, 716000, 717000, 718000, 719000, 720000, 721000, 722000, 723000, 724000, 725000, 726000, 727000, 728000, 729000, 730000, 731000, 732000, 733000, 734000, 735000, 736000, 737000, 738000, 739000, 740000, 741000, 742000, 743000, 744000, 745000, 746000, 747000, 748000, 749000, 750000, 751000, 752000, 753000, 754000, 755000, 756000, 757000, 758000, 759000, 760000, 761000, 762000, 763000, 764000, 765000, 766000, 767000, 768000, 769000, 770000, 771000, 772000, 773000, 774000, 775000, 776000, 777000, 778000, 779000, 780000, 781000, 782000, 783000, 784000, 785000, 786000, 787000, 788000, 789000, 790000, 791000, 792000, 793000, 794000, 795000, 796000, 797000, 798000, 799000, 800000, 801000, 802000, 803000, 804000, 805000, 806000, 807000, 808000, 809000, 810000, 811000, 812000, 813000, 814000, 815000, 816000, 817000, 818000, 819000, 820000, 821000, 822000, 823000, 824000, 825000, 826000, 827000, 828000, 829000, 830000, 831000, 832000, 833000, 834000, 835000, 836000, 837000, 838000, 839000, 840000, 841000, 842000, 843000, 844000, 845000, 846000, 847000, 848000, 849000, 850000, 851000, 852000, 853000, 854000, 855000, 856000, 857000, 858000, 859000, 860000, 861000, 862000, 863000, 864000, 865000, 866000, 867000, 868000, 869000, 870000, 871000, 872000, 873000, 874000, 875000, 876000, 877000, 878000, 879000, 880000, 881000, 882000, 883000, 884000, 885000, 886000, 887000, 888000, 889000, 890000, 891000, 892000, 893000, 894000, 895000, 896000, 897000, 898000, 899000, 900000, 901000, 902000, 903000, 904000, 905000, 906000, 907000, 908000, 909000, 910000, 911000, 912000, 913000, 914000, 915000, 916000, 917000, 918000, 919000, 920000, 921000, 922000, 923000, 924000, 925000, 926000, 927000, 928000, 929000, 930000, 931000, 932000, 933000, 934000, 935000, 936000, 937000, 938000, 939000, 940000, 941000, 942000, 943000, 944000, 945000, 946000, 947000, 948000, 949000, 950000, 951000, 952000, 953000, 954000, 955000, 956000, 957000, 958000, 959000, 960000, 961000, 962000, 963000, 964000, 965000, 966000, 967000, 968000, 969000, 970000, 971000, 972000, 973000, 974000, 975000, 976000, 977000, 978000, 979000, 980000, 981000, 982000, 983000, 984000, 985000, 986000, 987000, 988000, 989000, 990000, 991000, 992000, 993000, 994000, 995000, 996000, 997000, 998000, 999000]

In [ ]:
def compute_markout_pnl(df, market_df, horizons, symbol = None):
    market_data = market_df.copy()
    new_columns = {}

    for h in horizons:
        print(f"Processing horizon {h}ms...")

        markout_timestamps = df.timestamp + h

        market_prices = pd.merge_asof(
            pd.DataFrame({"timestamp": markout_timestamps}),
            market_data,
            on="timestamp",
            direction="backward" if h < 0 else "forward"
        )

        new_columns[f"future_mid_price_{h}_ms"] = market_prices["mid_price"]
        new_columns[f"open_markout_position_{h}_ms"] = df["signed_size"] * new_columns[f"future_mid_price_{h}_ms"]
        new_columns[f"markout_pnl_{h}_ms"] = new_columns[f"open_markout_position_{h}_ms"] - df["fill_value"]
        # computing margin as per the definition provided by instructions.txt
        new_columns[f"margin_{h}_ms"] = new_columns[f"markout_pnl_{h}_ms"] / df["size"] 
        
        print(f"""Debugging Symbol {symbol} at Horizon {h}ms:
        1. Market Prices (First 5) {market_prices.head()}
        2. Future Mid-Price (First 5)\n{new_columns[f"future_mid_price_{h}_ms"].head()}
        3. Open Markout Position (First 5)\n{new_columns[f"open_markout_position_{h}_ms"].head()}
        4. Markout PnL (First 5)\n{new_columns[f"markout_pnl_{h}_ms"].head()}
        5. Margin (First 5)\n{new_columns[f"margin_{h}_ms"].head()}
        """)

    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)

    return df


for symbol in merged_data:
    print(f"Processing {symbol}...")
    merged_data[symbol] = compute_markout_pnl(merged_data[symbol], market_data[symbol], horizon_ticks)

In [ ]:
for symbol in merged_data:
  print(f"Showing stats for symbol {symbol}")
  print(merged_data[symbol].head(3))

In [ ]:
for symbol, df in merged_data.items():
    merged_na = df.isna().any(axis=1)
    print(f"Shape for {symbol}: {df.shape}, NA Rows: {sum(merged_na)}")


In [ ]:
na_entries = {}

for symbol, df in merged_data.items():
    na_rows = df[df.isna().any(axis=1)]
    na_entries[symbol] = na_rows

for symbol, df_na in na_entries.items():
    print(f"NA Entries for {symbol}:")
    display(df_na.tail()) 

The last few rows of the data show that the `mid_price` is missing for some timestamps. This is because the market data is not available for these timestamps. To avoid contaminating the data, I will remove these rows from the dataset.

In [ ]:
for symbol, df in merged_data.items():
    merged_data[symbol] = df.dropna()
    print(f"Dropped NA rows for symbol: {symbol}")

### Part B: Visualizing Market and Trade Data for an Interesting One Hour Period and Aggregate Margin Curves

With cumulative pnl and markout pnl cumputed, I will now visualize the market and trade data for an interesting one hour period of my choice for each instrument. I will then plot the aggregate markout curve of all trades for each instrument using the intervals specified in the file "horizon_ticks".

#### Visualizing Market and Trade Data for an Interesting One Hour Period

Earlier, I plotted the trade pnl and cumulative pnl for each instrument. For each instrument, I noticed they shared a similar pattern where the `cumulative pnl` curve trends upward for `GMMAUSD` and `ZTAUSD` and downward for `BTAUSD` and `LMDAUSD` with an inflection point between timestamp `1 May 6pm` and `2 May 6am`. Moreover, the `trade pnl` curve for each instrument shows extreme spikes during this period. For my one-hour period, I will narrow down to the peak of the spikes to investigate the extreme spikes and gain a better understanding of what the trading strategy is doing during this period.

In [ ]:
def get_interesting_period(df, symbol):
    if symbol in ["BTAUSD", "LMDAUSD"]:
        # Get min cumulative PnL timestamp
        target_idx = df["pnl"].idxmin()
    elif symbol in ["GMMAUSD", "ZTAUSD"]:
        # Get max cumulative PnL timestamp
        target_idx = df["pnl"].idxmax()
    else:
        return None, None

    center_time = df.loc[target_idx, "timestamp"]
    
    # define 1-hour window in milliseconds (30 minutes before and after)
    start_time = center_time - (30 * 60 * 1000)  # 30 minutes back
    end_time = center_time + (30 * 60 * 1000)    # 30 minutes forward
    
    return start_time, end_time


In [ ]:
def plot_market_trades(df_market, df_trades, df_pnl, start_time, end_time, symbol):
    df_market_filtered = df_market[(df_market["timestamp"] >= start_time) & (df_market["timestamp"] <= end_time)]
    df_trades_filtered = df_trades[(df_trades["timestamp"] >= start_time) & (df_trades["timestamp"] <= end_time)]
    df_pnl_filtered = df_pnl[(df_pnl["timestamp"] >= start_time) & (df_pnl["timestamp"] <= end_time)]

    df_market_filtered["timestamp"] = pd.to_datetime(df_market_filtered["timestamp"], unit='ms')
    df_trades_filtered["timestamp"] = pd.to_datetime(df_trades_filtered["timestamp"], unit='ms')
    df_pnl_filtered["timestamp"] = pd.to_datetime(df_pnl_filtered["timestamp"], unit='ms')

    buys = df_trades_filtered[df_trades_filtered["side"] == "B"]
    sells = df_trades_filtered[df_trades_filtered["side"] == "S"]
    num_buys, num_sells = len(buys), len(sells)

    fig, ax1 = plt.subplots(figsize=(12, 6))

    ax1.plot(df_market_filtered["timestamp"], df_market_filtered["bid"], label="Bid Price", color="blue", alpha=0.6)
    ax1.plot(df_market_filtered["timestamp"], df_market_filtered["ask"], label="Ask Price", color="red", alpha=0.6)

    # visibility on buy and sell trades
    ax1.scatter(buys["timestamp"], buys["price"], label=f"Buys ({num_buys})", color="lime", marker="^", s=80, alpha=0.9, edgecolors="black", linewidth=1.2)
    ax1.scatter(sells["timestamp"], sells["price"], label=f"Sells ({num_sells})", color="magenta", marker="v", s=80, alpha=0.9, edgecolors="black", linewidth=1.2)

    ax1.set_xlabel("Time")
    ax1.set_ylabel("Price")
    ax1.set_title(f"Market Data, Trades, and Cumulative PnL for {symbol}\n({pd.to_datetime(start_time, unit='ms')} to {pd.to_datetime(end_time, unit='ms')})")
    ax1.legend(loc="upper left", bbox_to_anchor=(0, 1))
    ax1.grid()

    ax2 = ax1.twinx()
    ax2.plot(df_pnl_filtered["timestamp"], df_pnl_filtered["cumulative_pnl"], label="Cumulative PnL", color="brown", linestyle="dashed", linewidth=2)
    ax2.set_ylabel("Cumulative PnL")
    ax2.legend(loc="upper right", bbox_to_anchor=(1, 1))

    plt.xticks(rotation=45)
    plt.show()

In [ ]:
SYMBOL = "BTAUSD"

start_time, end_time = get_interesting_period(merged_data[SYMBOL], SYMBOL)

plot_market_trades(market_data[SYMBOL], trade_data[SYMBOL], merged_data[SYMBOL], start_time, end_time, SYMBOL)

The market data for BTAUSD shows a relatively wider bid-ask spread early in the period, with increased volatility between 23:00 - 23:20, leading to more frequent and dispersed trade executions. 

A closer examination of execution timing reveals potential latency issues. Between 22:50 and 23:00, sell orders were placed shortly after a price dip but executed during a price rally. Similarly, between 23:00 and 23:10, buy orders were executed shortly after a price increase but before a subsequent price drop, and sell orders were placed shortly after a price drop but during a rebound, further reinforcing the inefficiencies. This suggests that the strategy may be reacting to price movements with a delay, resulting in suboptimal trade execution.

Notably, after 23:20, despite a sustained price rally from 23:20 to 23:35, the strategy continued to execute sell orders while making no apparent buy trades. This lack of responsiveness to changing market trends may indicate either a rigid execution logic that fails to adapt to bullish conditions or a potential oversight in the strategy's risk management framework. These issues explain why the cumulative pnl curve trends downward over time.

In [ ]:
SYMBOL = "GMMAUSD"

start_time, end_time = get_interesting_period(merged_data[SYMBOL], SYMBOL)

plot_market_trades(market_data[SYMBOL], trade_data[SYMBOL], merged_data[SYMBOL], start_time, end_time, SYMBOL)

The market data for GMMAUSD shows an initial downward price movement, followed by a strong uptrend, before a slight reversal near the end of the period. The cumulative PnL exhibits a consistent upward trajectory throughout the entire period, indicating that the trading strategy effectively captured market trends and volatility without experiencing any major drawdowns. Moreover, trade execution is well-aligned with price movements, with buys (green triangles) occurring mostly at lower price levels and sells (purple triangles) at higher levels. This pattern is the opposite of what we observed in BTAUSD, where trade timing issues led to a steady decline in PnL.

Notably, near the end of the period, despite a price retracement, cumulative PnL continued to rise. This suggests that the strategy was either properly hedged or able to profit from the market correction, reinforcing its ability to adapt to changing market conditions. The sustained growth in PnL throughout both the downward and upward price movements highlights a robust execution strategy, capable of capitalizing on momentum while avoiding adverse selection or execution inefficiencies.

In [ ]:
SYMBOL = "LMDAUSD"

start_time, end_time = get_interesting_period(merged_data[SYMBOL], SYMBOL)

plot_market_trades(market_data[SYMBOL], trade_data[SYMBOL], merged_data[SYMBOL], start_time, end_time, SYMBOL)

Early in the period, the bid-ask spread is extremely tight, and during this time, cumulative PnL remains relatively stable. However, around 22:40, the spread begins to widen, signaling increased market volatility. At this point, cumulative PnL starts to decline, indicating that the strategy may not have adapted well to the changing market conditions.

One key observation is that the strategy executed more sell trades than buy trades during the period when the price was trending upwards. This suggests that the strategy was either expecting a reversal that never materialized or was caught in an adverse selection scenario, where its sell orders were consistently being executed before the price moved higher. 

Near the end of the period, despite a moderate upward price trend, the strategy continued to sell more than buy, reinforcing the negative PnL trajectory. This suggests that the strategy either failed to adjust to market conditions or was operating under a trading logic that did not account for sustained price increases. The timing of these trades, especially the continued selling into an uptrend, points to a potential misalignment between strategy assumptions and real-time market behavior, which contributed to the continued decline in cumulative PnL.

In [ ]:
SYMBOL = "ZTAUSD"

start_time, end_time = get_interesting_period(merged_data[SYMBOL], SYMBOL)

plot_market_trades(market_data[SYMBOL], trade_data[SYMBOL], merged_data[SYMBOL], start_time, end_time, SYMBOL)

The market data for ZTAUSD shows a steady price increase, followed by periods of volatility and pullbacks, before ultimately continuing its upward trend. The cumulative PnL exhibits a strong and consistent increase throughout the period, suggesting that the strategy was able to capitalize on both upward trends and price reversals. Unlike BTAUSD and LMDAUSD, where PnL steadily declined due to poor trade execution or misalignment with market trends, ZTAUSD's strategy appears to have effectively adjusted to price swings.

Observing the trade execution patterns, I noticed that there are instances where buys occur at local price peaks, which may indicate aggressive momentum chasing rather than waiting for optimal entry points. Moreover, it seems to be that the strategy successfully averaged down by continuing to buy as the price declined, ultimately benefiting when the price rebounded. This suggests a potential mean-reversion component within the strategy. However, to determine whether this buy signal was based on skillful execution or simply favorable market conditions.

#### Visualizing Aggregate Markout Curves

Next, I will plot the aggregate markout curve of all trades for each instrument using the intervals specified in the file "horizon_ticks". To accomplish this, I computed the average margin of all trades at each interval.

In [ ]:
def plot_aggregate_markout_margin(df, horizons, symbol):
    aggregate_markout_margin = {
        h: df[f"margin_{h}_ms"].mean() for h in horizons
    }

    plt.figure(figsize=(8, 5))
    plt.plot(aggregate_markout_margin.keys(), aggregate_markout_margin.values(), marker='o', linestyle='-', color='red', label="Aggregate Markout Margin")

    plt.xlabel("Time Horizon (ms)")
    plt.ylabel("PNL Margin (Normalized by Volume)")
    plt.title(f"Aggregate Markout Margin Curve for {symbol}")
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
SYMBOL = "BTAUSD"
plot_aggregate_markout_margin(merged_data[SYMBOL], horizon_ticks, SYMBOL)

For `BTAUSD`, the markout margin starts positive but quickly turns negative within the first few milliseconds and remains negative for most of the horizon. This suggests that trades initially look profitable but soon face adverse selection, with prices moving against the trade shortly after execution. The negative signal persists for a prolonged period, indicating sustained execution inefficiencies.

In [ ]:
SYMBOL = "GMMAUSD"
plot_aggregate_markout_margin(merged_data[SYMBOL], horizon_ticks, SYMBOL)

For `GMMAUSD`, the markout margin starts high and gradually declines before stabilizing into a consistent upward trend. This suggests that the strategy benefits from favorable execution over time, with positions improving in value as the market moves in the intended direction. The signal remains strong throughout the horizon, indicating sustained profitability.

In [ ]:
SYMBOL = "LMDAUSD"
plot_aggregate_markout_margin(merged_data[SYMBOL], horizon_ticks, SYMBOL)

For `LMDAUSD`, the markout margin starts deeply negative and remains so for most of the time horizon, with minor fluctuations. This suggests that the strategy consistently faces price movements against its trades post-execution, likely due to poor execution timing or adverse selection. The signal remains weak, indicating sustained underperformance.



In [ ]:
SYMBOL = "ZTAUSD"
plot_aggregate_markout_margin(merged_data[SYMBOL], horizon_ticks, SYMBOL)

For `ZTAUSD`, the markout margin initially dips negative but recovers after a short period, stabilizing around neutral to slightly positive levels. This suggests that while trades may initially experience adverse selection, the strategy manages to recover value over time. The signal does not persist strongly, indicating a mixed or time-dependent effectiveness of execution.

### Part C: Further Investigation if the Strategy is a Maker or Taker, Interesting Characteristics of the Strategy, and Maximum Drawdown. 

Finally, I will investigate whether the strategy is a maker or taker by analyzing the trade execution prices relative to the mid-price at the time of execution. I will also identify any interesting characteristics of the strategy based on the visualizations and analysis conducted earlier. Additionally, I will calculate the maximum drawdown for each instrument to assess the strategy's risk profile, and at which timestamp it occurred.

##### Maker or Taker Analysis

To determine whether the strategy is a maker or taker, I compared the execution prices of trades to the mid-price at the time of execution. If the execution price is better than the mid-price, the strategy is considered a maker, while if the execution price is worse than the mid-price, the strategy is considered a taker. This heuristic may not be perfect, but it is a best effort given the available data.

In [ ]:
def classify_maker_taker(merged_data, tolerance_factor=10):
    df = merged_data.copy()
    
    # Dictionary of tick sizes by symbol
    tick_sizes = {
        "BTAUSD": 1e-05,
        "GMMAUSD": 5e-05,
        "LMDAUSD": 1e-05,
        "ZTAUSD": 5e-05
    }

    tick_size = tick_sizes[symbol]
    tolerance = tolerance_factor * tick_size
    
    # Here assume that bid and ask are BBOs
    taker_conditions = (
        # A taker buys at or above ask (with tolerance)
        ((df["side"] == "buy") & (df["price"] >= df["ask"] - tolerance)) |
        # A taker sells at or below bid (with tolerance)  
        ((df["side"] == "sell") & (df["price"] <= df["bid"] + tolerance)) 
    )

    df["maker_taker"] = np.where(taker_conditions, "taker", "maker")

    maker_count = (df["maker_taker"] == "maker").sum()
    taker_count = (df["maker_taker"] == "taker").sum()

    print(f"Symbol: {symbol}, Tick Size: {tick_size}")
    print(f"Makers: {maker_count}, Takers: {taker_count}")
    
    return df

In [ ]:
for symbol in merged_data:
    print(f"Classifying maker/taker for {symbol}...")
    merged_data[symbol] = classify_maker_taker(merged_data[symbol])

for symbol, df in merged_data.items():
    merged_na = df.isna().any(axis=1)
    print(f"Shape for {symbol}: {df.shape}, NA Rows: {sum(merged_na)}")

The strategy is classified as a `Maker` for all instruments. I determined this by evaluating whether the trade was a `Taker`, which is if buy fill prices were greater or equal to the `ask` price or if sell fill prices were less than or equal to the `bid` price. Here, I assumed that the bid and ask columns represent the best `bid` and `offer` (BBO) in the order book.

##### Interesting Characteristics of the Strategy

After determining that the strategy is a `Maker`, I will compute some interesting characteristics I believe are worth mentioning.

In [ ]:
def compute_post_trade_characteristics(merged_data, market_df, symbol):
    df = merged_data.copy()

    total_trades = len(df)
    print(f"Total Trades: {total_trades}")

    avg_trade_size = df["size"].mean()
    print(f"Average Trade Size: {avg_trade_size:.2f}")

    total_pnl = df["cumulative_pnl"].iloc[-1]
    print(f"Total PnL: {total_pnl:.2f}")

    avg_pnl_per_trade = total_pnl / total_trades if total_trades > 0 else 0
    print(f"Average PnL per Trade: {avg_pnl_per_trade:.2f}")

    # total accumulated inventory (net position)
    total_inventory = df["signed_size"].sum()
    print(f"Total Accumulated Inventory: {total_inventory:.2f}")

    # average inventory
    avg_inventory = df["signed_size"].mean()
    print(f"Average Inventory: {avg_inventory:.2f}")

    df['holding_time'] = df['timestamp'].diff()
    avg_holding_time = df['holding_time'].mean()
    print(f"Average Holding Time per Trade: {avg_holding_time:.2f} ms")
    
    # trade frequency (trades per second)
    trade_duration = (df["timestamp"].max() - df["timestamp"].min()) / 1000  # Convert ms to seconds
    trade_frequency = total_trades / trade_duration if trade_duration > 0 else 0
    print(f"Trade Frequency: {trade_frequency:.6f} trades/s")

    # quote aggressiveness (average distance from mid-price)
    df["quote_aggressiveness"] = abs(df["price"] - df["mid_price"])
    avg_quote_aggressiveness = df["quote_aggressiveness"].mean()
    print(f"Quote Aggressiveness (Avg Distance from Mid-Price): {avg_quote_aggressiveness:.5f}")
    avg_quote_aggressiveness_ratio = avg_quote_aggressiveness / metadata[symbol]["tick_size"]
    print(f"Quote Aggressiveness Ratio (Avg Distance from Mid-Price to Tick Size): {avg_quote_aggressiveness_ratio:.5f}")

    # execution skew (buy vs sell trade ratio)
    buy_trades = (df["side"] == "B").sum()
    sell_trades = (df["side"] == "S").sum()
    execution_skew = buy_trades / sell_trades
    print(f"Execution Skew (Buy/Sell Trade Ratio): {execution_skew:.2f}")

    # calculate market impact defined as the change in mid price 15 seconds after a trade is fulfilled
    df["timestamp_future"] = df["timestamp"] + 15000  # add 15 seconds minute (15000 ms) to trade timestamp
    df = pd.merge_asof(df, market_df[["timestamp", "mid_price"]], left_on="timestamp_future", right_on="timestamp", direction="forward", suffixes=("", "_future"))

    df["market_impact"] = df["mid_price_future"] - df["mid_price"]
    avg_market_impact = df["market_impact"].mean()
    print(f"Average Market Impact (15 sec after trade): {avg_market_impact:.10f}")

    avg_market_impact_over_tick = avg_market_impact / metadata[symbol]["tick_size"]
    print(f"Average Market Impact Over Tick Size Ratio (15 sec after trade): {avg_market_impact_over_tick:.5f}")
    

In [ ]:
SYMBOL = "BTAUSD"

compute_post_trade_characteristics(merged_data[SYMBOL], market_data[SYMBOL], SYMBOL)

For `BTAUSD`, the strategy executed 2,959 trades with an average trade size of 5,489.02, maintaining a net short position (-264,000 units). The average PnL per trade is negative (-0.28), suggesting execution inefficiencies or adverse selection. The holding time per trade is 58 seconds, indicating short-term positioning.

With a trade frequency of 0.017 trades/sec, the strategy operates at a moderate pace. The average quote distance from `mid_price` is 0.00005, and as a ratio of tick size, it is 5.17759.The execution skew (0.99) indicates a balanced execution profile, with a slight bias toward selling.

The market impact over 15 seconds is minimal (0.0000027), equating to 0.27 ticks on average. Given the negative PnL, further investigation into order placement, fill quality, and spread dynamics is necessary to assess whether the strategy is experiencing adverse selection or suffering from suboptimal execution timing.

In [ ]:
SYMBOL = "GMMAUSD"

compute_post_trade_characteristics(merged_data[SYMBOL], market_data[SYMBOL], SYMBOL)

For `GMMAUSD`, the strategy executed 2,949 trades with an average trade size of 5,488.64, maintaining a net long position (110,000 units). The average PnL per trade is positive (1.37), indicating profitable execution and effective order placement. The holding time per trade is 58 seconds, suggesting short-term positioning.

With a trade frequency of 0.017 trades/sec, the strategy operates at a moderate pace. The average quote distance from `mid_price` is 0.00025, and as a ratio of tick size, it is 5.06646. The execution skew (0.98) indicates a balanced execution profile.

The market impact over 15 seconds (0.0000466) equates to 0.93 ticks per trade, meaning the strategy has a moderate effect on price movements. While the higher market impact suggests increased order urgency, the positive PnL indicates that the strategy effectively capitalized on market conditions.


In [ ]:
SYMBOL = "LMDAUSD"

compute_post_trade_characteristics(merged_data[SYMBOL], market_data[SYMBOL], SYMBOL)

For `LMDAUSD`, the strategy executed 3,149 trades with an average trade size of 5,493.49, maintaining a net long position (683,000 units). The average PnL per trade is negative (-0.27), suggesting execution inefficiencies or adverse selection. The holding time per trade is 54 seconds, indicating short-term positioning.

With a trade frequency of 0.018 trades/sec, the strategy operates at a moderate pace. The average quote distance from `mid_price` is 0.00005, and as a ratio of tick size, it is 5.10225. The execution skew (1.07) suggests a slight bias toward buying.

The market impact over 15 seconds is negative (-0.0000085), equating to -0.85 ticks per trade, implying that trades tend to be followed by adverse price movements. The negative PnL combined with unfavorable market impact suggests that the strategy may be experiencing adverse selection or liquidity-taking inefficiencies for this currency pair.

In [ ]:
SYMBOL = "ZTAUSD"

compute_post_trade_characteristics(merged_data[SYMBOL], market_data[SYMBOL], SYMBOL)

For `ZTAUSD`, the strategy executed 2,960 trades with an average trade size of 5,487.50, maintaining a net short position (-655,000 units). The average PnL per trade is positive (1.40), indicating profitable execution and effective order placement. The holding time per trade is 58 seconds, suggesting short-term positioning.

With a trade frequency of 0.017 trades/sec, the strategy operates at a moderate pace. The average quote distance from `mid_price` is 0.00026, and as a ratio of tick size, it is 5.16689. The execution skew (0.96) indicates a balanced execution profile.

The market impact over 15 seconds is negative (-0.0000511), equating to -1.02 ticks per trade, implying that trades are followed by adverse price movements. While the PnL remains positive, the negative market impact suggests that the strategy's executions may be causing price movements against its positions, potentially due to liquidity demand or hidden adverse selection effects.

##### Computing the Maximum Drawdown and Finding When Maximum Drawdown Occurred

Finally, I will compute the maximum drawdown for each instrument and identify when the maximum drawdown occurred.

In [ ]:
def compute_max_drawdown(df):

    rolling_max_pnl = df["cumulative_pnl"].cummax()

    drawdown = rolling_max_pnl - df["cumulative_pnl"]

    max_drawdown = drawdown.max()
    max_drawdown_idx = drawdown.idxmax()
    max_drawdown_timestamp = df.loc[max_drawdown_idx, "timestamp"]
    max_drawdown_datetime = pd.to_datetime(max_drawdown_timestamp, unit="ms")

    print(f"Maximum Drawdown: {max_drawdown:.2f}")
    print(f"Timestamp (ms): {max_drawdown_timestamp}")
    print(f"Timestamp (datetime): {max_drawdown_datetime}")
    

In [ ]:
SYMBOL = "BTAUSD"

compute_max_drawdown(merged_data[SYMBOL])

The maximum drawdown experienced by the strategy for `BTAUSD` was 826.81, occurring at timestamp 1683067370875 (2023-05-02 22:42:50.875 UTC).

In [ ]:
SYMBOL = "GMMAUSD"

compute_max_drawdown(merged_data[SYMBOL])

The maximum drawdown experienced by the strategy for `GMMAUSD` was 1.30, occurring at timestamp 1683010823350 (2023-05-02 07:00:23.350 UTC).

In [ ]:
SYMBOL = "LMDAUSD"

compute_max_drawdown(merged_data[SYMBOL])

The maximum drawdown experienced by the strategy for `LMDAUSD` was 861.79, occurring at timestamp 1683067273125 (2023-05-02 22:41:13.125 UTC).

In [ ]:
SYMBOL = "ZTAUSD"

compute_max_drawdown(merged_data[SYMBOL])

The maximum drawdown experienced by the strategy for `ZTAUSD` was 0.90, occurring at timestamp 1682951827050 (2023-05-01 14:37:07.050 UTC).